In [2]:
import numpy as np
import pandas as pd
from mne.io import RawArray
from mne.channels import read_montage
from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs, concatenate_raws, pick_types
from mne.decoding import CSP

from sklearn.linear_model import LogisticRegression
from glob import glob

from scipy.signal import butter, lfilter, convolve, boxcar

def creat_mne_raw_object(fname,read_events=True):
    """Create a mne raw instance from csv file"""
    # Read EEG file
    data = pd.read_csv(fname)
    
    # get chanel names
    ch_names = list(data.columns[1:])
    
    # read EEG standard montage from mne
    montage = read_montage('standard_1005',ch_names)

    ch_type = ['eeg']*len(ch_names)
    data = 1e-6*np.array(data[ch_names]).T # From microvolts to volts
    
    if read_events:
        # events file
        ev_fname = fname.replace('_data','_events')
        # read event file
        events = pd.read_csv(ev_fname)
        events_names = events.columns[1:]
        events_data = np.array(events[events_names]).T
        
        # define channel type, the first is EEG, the last 6 are stimulations
        ch_type.extend(['stim']*6)
        ch_names.extend(events_names)
        # concatenate event file and data
        data = np.concatenate((data,events_data))
        
    # create and populate MNE info structure
    info = create_info(ch_names,sfreq=500.0, ch_types=ch_type, montage=montage)
    info['filename'] = fname
    
    # create raw object 
    raw = RawArray(data,info,verbose=False)
    
    return raw

subjects = range(1,2)
ids_tot = []

# design a butterworth bandpass filter 
freqs = [7, 30]
b,a = butter(5,np.array(freqs)/250.0,btype='bandpass')

# convolution
# window for smoothing features
nwin = 250

# training subsample
subsample = 10

cols = ['HandStart','FirstDigitTouch',
        'BothStartLoadPhase','LiftOff',
        'Replace','BothReleased']

for subject in subjects:

    ################ READ DATA ################################################
    fnames =  glob('/Users/eszti/Documents/__NYC_DSA/Dev/capstone_eeg/train/subj%d_series*_data.csv' % (subject))
    
    # read and concatenate all the files
    raw = concatenate_raws([creat_mne_raw_object(fname) for fname in fnames])
       
    # pick eeg signal
    picks = pick_types(raw.info,eeg=True)
    
    # Filter data for alpha frequency and beta band
    # Note that MNE implement a zero phase (filtfilt) filtering not compatible
    # with the rule of future data.
    # Here we use left filter compatible with this constraint. 
    raw._data[picks] = lfilter(b,a,raw._data[picks])
    raw.df = raw.to_data_frame(index = None)
    raw.df.to_csv('raw_filtered.csv', float_format='%.5f')

Converting time column to int64...


In [7]:
df

signal,Fp1,Fp2,F7,F3,Fz,F4,F8,FC5,FC1,FC2,...,O1,Oz,O2,PO10,HandStart,FirstDigitTouch,BothStartLoadPhase,LiftOff,Replace,BothReleased
time,,,,,,,,,,,,,,,,,,,,,
0,-0.001265,0.014813,0.008611,0.004938,0.008611,0.000612,0.029260,0.011386,0.001428,0.006448,...,0.018731,0.007060,0.004897,0.028729,0,0,0,0,0,0
2,-0.012461,0.146013,0.085575,0.049038,0.085820,0.013619,0.285121,0.114924,0.014488,0.064253,...,0.183672,0.068690,0.047042,0.286186,0,0,0,0,0,0
4,-0.066333,0.706775,0.414775,0.238879,0.421733,0.117207,1.363546,0.569992,0.071189,0.316153,...,0.888771,0.329184,0.221490,1.403162,0,0,0,0,0,0
6,-0.257894,2.272853,1.319253,0.772359,1.381865,0.588425,4.330544,1.877478,0.229596,1.039294,...,2.873506,1.052265,0.692015,4.577425,0,0,0,0,0,0
8,-0.805919,5.536024,3.123502,1.891371,3.438019,2.042638,10.420886,4.684159,0.556375,2.601130,...,7.079654,2.561244,1.637996,11.319675,0,0,0,0,0,0
10,-2.092624,11.001224,5.912823,3.783841,6.987336,5.393853,20.497131,9.557551,1.097676,5.336563,...,14.323241,5.123012,3.171634,22.862117,0,0,0,0,0,0
12,-4.604808,18.674025,9.379877,6.486144,12.141701,11.531404,34.577631,16.769529,1.857916,9.399681,...,24.937794,8.839102,5.279218,39.542185,0,0,0,0,0,0
14,-8.745053,27.867577,12.916432,9.827689,18.587324,20.859719,51.613692,26.184647,2.794176,14.647395,...,38.521434,13.579974,7.810719,60.432659,0,0,0,0,0,0
16,-14.563086,37.222207,15.919824,13.441653,25.578169,32.983483,69.547362,37.253773,3.823863,20.591731,...,53.847065,18.963697,10.503109,83.312786,0,0,0,0,0,0
